In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import * 
from pyspark.sql import SparkSession, DataFrame as SparkDataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import col,isnan, when, count
from pyspark import SparkContext
from pyspark.sql import SQLContext
import json
from functools import reduce
import sys
from cassandra.cluster import Cluster
import os
# from mock.tasks import adiciona_carro
cluster = Cluster(['cassandra'])
session = cluster.connect()

ss = SparkSession.builder.appName("test").getOrCreate()
sql = SQLContext(ss)

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
session.execute("USE simulacao")

In [4]:
session.execute("select count(*) from simulacao")[0]

/tmp/ipykernel_16086/795349805.py:1: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  session.execute("select count(*) from simulacao")[0]


Row(count=3712)

In [42]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = 142 ALLOW FILTERING;")

In [43]:
result[0]

/tmp/ipykernel_16086/223460175.py:1: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  result[0]


Row(rodovia='BR-116', tempo_da_simulacao=142, placa='ARG5B67', pos_x=655.0, pos_y=168.99999999999872)

In [44]:
r = list(result)

In [45]:
r

[Row(rodovia='BR-116', tempo_da_simulacao=142, placa='ARG5B67', pos_x=655.0, pos_y=168.99999999999872),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='CHI6S78', pos_x=745.0, pos_y=820.0),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='ECU6I78', pos_x=565.0, pos_y=90.75),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='GUY1X23', pos_x=385.0, pos_y=310.25000000000017),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='PAR7P89', pos_x=475.0, pos_y=152.25000000000003),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='SUR8W90', pos_x=745.0, pos_y=749.15),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='URU0A12', pos_x=835.0, pos_y=650.5),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='URU4O56', pos_x=565.0, pos_y=869.2500000000007)]

In [46]:
df = ss.createDataFrame(r)

df.show()

+-------+------------------+-------+-----+------------------+
|rodovia|tempo_da_simulacao|  placa|pos_x|             pos_y|
+-------+------------------+-------+-----+------------------+
| BR-116|               142|ARG5B67|655.0|168.99999999999872|
| BR-116|               142|CHI6S78|745.0|             820.0|
| BR-116|               142|ECU6I78|565.0|             90.75|
| BR-116|               142|GUY1X23|385.0|310.25000000000017|
| BR-116|               142|PAR7P89|475.0|152.25000000000003|
| BR-116|               142|SUR8W90|745.0|            749.15|
| BR-116|               142|URU0A12|835.0|             650.5|
| BR-116|               142|URU4O56|565.0| 869.2500000000007|
+-------+------------------+-------+-----+------------------+



In [47]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = 143 ALLOW FILTERING;")
r = list(result)
df2 = ss.createDataFrame(r)
df2.show()

+-------+------------------+-------+-----+------------------+
|rodovia|tempo_da_simulacao|  placa|pos_x|             pos_y|
+-------+------------------+-------+-----+------------------+
| BR-116|               143|ARG5B67|655.0|162.99999999999872|
| BR-116|               143|CHI6S78|745.0|             776.0|
| BR-116|               143|ECU6I78|565.0|325.35000000000014|
| BR-116|               143|GUY1X23|385.0| 435.1000000000003|
| BR-116|               143|PAR7P89|475.0| 268.8000000000002|
| BR-116|               143|SUR8W90|745.0|            749.15|
| BR-116|               143|URU0A12|835.0|             614.5|
| BR-116|               143|URU4O56|565.0| 904.5000000000007|
| BR-116|               143|URU6M78|655.0|            826.75|
+-------+------------------+-------+-----+------------------+



In [86]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = 144 ALLOW FILTERING;")
r = list(result)
df25 = ss.createDataFrame(r)
df25.show()

+-------+------------------+-------+-----+------------------+
|rodovia|tempo_da_simulacao|  placa|pos_x|             pos_y|
+-------+------------------+-------+-----+------------------+
| BR-116|               144|ARG5B67|655.0|146.99999999999872|
| BR-116|               144|CHI6S78|745.0|             776.0|
| BR-116|               144|ECU6I78|565.0|245.55000000000007|
| BR-116|               144|GUY1X23|385.0|459.75000000000034|
| BR-116|               144|PAR7P89|475.0| 317.5500000000002|
| BR-116|               144|SUR8W90|745.0|             753.5|
| BR-116|               144|URU0A12|835.0|             658.5|
| BR-116|               144|URU4O56|565.0| 800.5500000000006|
| BR-116|               144|URU6M78|745.0|            800.75|
+-------+------------------+-------+-----+------------------+



In [59]:
df_empty = ss.createDataFrame([], "placa: string, posicao: int, tempo_da_simulacao: int, velocidade: double, aceleracao: double, posicao_prevista: double")

In [78]:
def differenciate(val1, val2, t1, t2):
    return (val2 - val1)/(t2 - t1)



def processa_carro(DadosNovos, DadosCarros, colision_tolerance, colision_tolerance_quad):
    DadosNovos = DadosNovos.select(F.col('pos_y').alias('posicao_nova'),
                      F.col('tempo_da_simulacao').alias('tempo_da_simulacao_novo'),F.col('placa'))
    data_joined = DadosCarros.join(DadosNovos, on="placa", how='right')
    data_joined = data_joined.withColumn("diferenca_de_posicao", col("posicao_nova") - col("posicao"))
    data_joined = data_joined.withColumn("diferenca_de_horario", col("tempo_da_simulacao_novo") - col("tempo_da_simulacao"))
    data_joined = data_joined.withColumnRenamed("velocidade", "velocidade_antiga")
    data_joined = data_joined.withColumn("velocidade", col("diferenca_de_posicao") / col("diferenca_de_horario"))
    data_joined = data_joined.withColumn("diferenca_de_velocidade", col("velocidade") - col("velocidade_antiga"))
    data_joined = data_joined.withColumn("aceleracao", col("diferenca_de_velocidade") / col("diferenca_de_horario"))
    data_joined = data_joined.withColumn("aceleracao", col("diferenca_de_velocidade") / col("diferenca_de_horario"))
    data_joined = data_joined.drop("velocidade_antiga", "posicao", "tempo_da_simulacao",
                     "diferenca_de_posicao", "diferenca_de_horario",
                     "diferenca_de_velocidade", "posicao_prevista")
    data_joined = data_joined.withColumnRenamed("posicao_nova", "posicao")
    data_joined = data_joined.withColumnRenamed("tempo_da_simulacao_novo", "tempo_da_simulacao")
    data_joined = data_joined.withColumn("posicao_prevista", col("posicao")\
                           + col("velocidade")*colision_tolerance\
                           + col("aceleracao")*colision_tolerance_quad)
    return data_joined
    

In [87]:
df3 = processa_carro(df, df_empty, 1, )

In [88]:
df3 = processa_carro(df2, df3, 10, 50)

In [89]:
df4 = processa_carro(df25, df3, 10, 50).show()

+-------+-------------------+------------------+------------------+-------------------+-------------------+
|  placa|         aceleracao|           posicao|tempo_da_simulacao|         velocidade|   posicao_prevista|
+-------+-------------------+------------------+------------------+-------------------+-------------------+
|ARG5B67|              -10.0|146.99999999999872|               144|              -16.0| -513.0000000000013|
|CHI6S78|               44.0|             776.0|               144|                0.0|             2976.0|
|ECU6I78| -314.4000000000002|245.55000000000007|               144| -79.80000000000007|-16272.450000000012|
|GUY1X23| -100.2000000000001|459.75000000000034|               144| 24.650000000000034| -4303.750000000005|
|PAR7P89| -67.80000000000015| 317.5500000000002|               144|              48.75|-2584.9500000000075|
|SUR8W90|  4.350000000000023|             753.5|               144|  4.350000000000023| 1014.5000000000014|
|URU0A12|               80.0